<a href="https://colab.research.google.com/github/AleksxanderAlekseyuk/Deal_with_epicrisis/blob/main/treatment_initiation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import re
import os
import pandas as pd
import numpy as np
from glob import glob
from re import sub

In [24]:
lst = list(glob(os.path.join('/content/drive/MyDrive/phd/script.zip (Unzipped Files)/script', '*')))
ids = []
birthday = []
treatment = []

In [25]:
for doc in lst:
   with open(doc, encoding='utf-8', errors='ignore') as f:
     file_ = sub('<|>|\\(|\\)|меньше|меньшн|больше|более|от|менее|iultra|\\(|ручная|\\)|\\(|автоматическая|\\)', 
                  '.', ''.join(f.read().split()).lower())
     file_ =  sub(r'(0?[1-9]|[12]\d|30|31)[.](0?[1-9]|1[0-2])[.](\d{4})', 
                     '\\1-\\2-\\3', file_)
     file_ = sub(r'(0?[1-9]|[12]\d|30|31)[.](0?[1-9]|1[0-2])[.](\d{2})',
                    '\\1-\\2-20\\3', file_)
     id = re.findall('\d+', file_)[0]
     birthday_ = re.findall('\d{2}-\d{2}-\d{4}', file_)[0]
     treatment_ = re.findall("(?<=проведенноелечение:).*?(?=рекомендации|наблюдение)", 
                            file_)
     ids.append(id)
     birthday.append(birthday_)
     treatment.append(treatment_)

In [26]:
ids[:10]

['27389',
 '27052',
 '27071',
 '887',
 '3',
 '27027',
 '1',
 '27186',
 '27042',
 '27222']

In [50]:
data_treatment = {'ID':ids, 'birthday':birthday,'treatment':treatment}
df = pd.DataFrame(data_treatment)
df.replace(r'', np.nan, regex=True)

,ID,birthday,treatment
0,27389,15-09-1952,"[пронпозиция,кислород.ерапия,дексаметазон,омеп..."
1,27052,06-03-1952,"[инфузионнаятерапия,витамин.ерапия,гепарин,дек..."
2,27071,18-07-1955,"[бортезамиб,циклофосфан,дексаметазонв\в,фуросе..."
3,887,18-09-1984,"[пронпозиция,дексаметазон,омепразол,фраксипари..."
4,3,20-02-1967,[]
...,...,...,...
1142,26724,12-08-1978,"[дексаметазон,фрагмин,гепарин,преднизолон,ацц.]"
1143,26637,11-04-1958,"[дексаметазон,гепарин,амбраксол,омепразол,эуфи..."
1144,26639,27-05-1952,"[пронпозиция,гепарин,ацц,дексаметазон,омепразо..."
1145,26636,26-07-1968,"[пронпозиция,кислород.ерапия,дексаметазон,натр..."


In [32]:
df.head()

,ID,birthday,treatment
0,27389,15-09-1952,"[пронпозиция,кислород.ерапия,дексаметазон,омеп..."
1,27052,06-03-1952,"[инфузионнаятерапия,витамин.ерапия,гепарин,дек..."
2,27071,18-07-1955,"[бортезамиб,циклофосфан,дексаметазонв\в,фуросе..."
3,887,18-09-1984,"[пронпозиция,дексаметазон,омепразол,фраксипари..."
4,3,20-02-1967,[]


In [28]:
df.to_csv("treatment_raw_edition.csv")

In [51]:
df["Treatment"] = df["treatment"].apply(lambda x: ' '.join(dict.fromkeys(x).keys()))

In [53]:
df["Treatment"] = df["Treatment"].values.astype('str')

In [54]:
df["Treatment"]

0       пронпозиция,кислород.ерапия,дексаметазон,омепр...
1       инфузионнаятерапия,витамин.ерапия,гепарин,декс...
2       бортезамиб,циклофосфан,дексаметазонв\в,фуросем...
3       пронпозиция,дексаметазон,омепразол,фраксипарин...
4                                                        
                              ...                        
1142        дексаметазон,фрагмин,гепарин,преднизолон,ацц.
1143    дексаметазон,гепарин,амбраксол,омепразол,эуфил...
1144    пронпозиция,гепарин,ацц,дексаметазон,омепразол...
1145    пронпозиция,кислород.ерапия,дексаметазон,натри...
1146    омепразол,дексаметазон,амброксолацецезон,гепар...
Name: Treatment, Length: 1147, dtype: object

In [46]:
import re
import nltk
import string
PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

df["Treatment"] = df["Treatment"].apply(lambda text: remove_punctuation(text))


In [55]:
stopwords = "свой, витс, ммольл, крови,ммольл,в,по,18122020,время, вв"

     

In [56]:
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in \
                     stopwords])
df["Treatment"] = df["Treatment"].apply(lambda text: remove_stopwords(text))

In [59]:
df["Treatment"]

0       пронпозиция,кислород.ерапия,дексаметазон,омепр...
1       инфузионнаятерапия,витамин.ерапия,гепарин,декс...
2       бортезамиб,циклофосфан,дексаметазонв\в,фуросем...
3       пронпозиция,дексаметазон,омепразол,фраксипарин...
4                                                        
                              ...                        
1142        дексаметазон,фрагмин,гепарин,преднизолон,ацц.
1143    дексаметазон,гепарин,амбраксол,омепразол,эуфил...
1144    пронпозиция,гепарин,ацц,дексаметазон,омепразол...
1145    пронпозиция,кислород.ерапия,дексаметазон,натри...
1146    омепразол,дексаметазон,амброксолацецезон,гепар...
Name: Treatment, Length: 1147, dtype: object

In [ ]:
from collections import Counter
cnt = Counter()
for text in df["Treatment"].values:
    for word in text.split():
        cnt[word] += 1
        
cnt.most_common(100)

In [63]:
tt = df["Treatment"][:5]

In [69]:
for word in tt.values.split():
  print(word)

AttributeError: ignored